# Performance Reporting Example

This notebook demonstrates how to use the new `PerformanceReport` module to generate standardized metrics and charts.

In [ ]:
import sys
sys.path.insert(0, '../')

from backt import Backtester, BacktestConfig
from backt.reporting import PerformanceReport, ReportConfig
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Run a Backtest

First, let's run a simple moving average crossover strategy.

In [ ]:
# Define strategy
def ma_crossover_strategy(market_data, current_time, positions, context, params):
    """Simple moving average crossover strategy"""
    orders = {}
    
    for symbol, data in market_data.items():
        if len(data) < params['slow_window']:
            continue
        
        # Calculate MAs
        fast_ma = data['close'].rolling(params['fast_window']).mean().iloc[-1]
        slow_ma = data['close'].rolling(params['slow_window']).mean().iloc[-1]
        
        # Generate signals
        if fast_ma > slow_ma:
            orders[symbol] = {'action': 'target_weight', 'weight': 0.5}
        else:
            orders[symbol] = {'action': 'target_weight', 'weight': 0.0}
    
    return orders

In [ ]:
# Configure backtest
backtest_params = {
    'start_date': '2020-01-01',
    'end_date': '2023-12-31',
    'initial_capital': 100000,
    'symbols': ['AAPL', 'MSFT']
}

strategy_params = {
    'fast_window': 20,
    'slow_window': 50
}

config = BacktestConfig(
    start_date=backtest_params['start_date'],
    end_date=backtest_params['end_date'],
    initial_capital=backtest_params['initial_capital']
)

# Run backtest
backtester = Backtester(config)
result = backtester.run(
    strategy_function=ma_crossover_strategy,
    symbols=backtest_params['symbols'],
    strategy_params=strategy_params
)

print("✅ Backtest complete!")

## 2. Basic Usage - Default Report

Generate a complete report with all default sections.

In [ ]:
# Create report with defaults
report = PerformanceReport(result, initial_capital=backtest_params['initial_capital'])

# Print formatted text report
report.print_report()

In [ ]:
# Show all charts
report.show_charts()

## 3. Customized Report Configuration

Create a custom report with specific sections and verbose metrics.

In [ ]:
# Configure custom report
custom_config = ReportConfig(
    include_returns=True,
    include_risk=True,
    include_trading=True,
    include_portfolio=True,
    include_assessment=True,
    verbose_metrics=True,  # Show all available metrics
    
    # Chart options
    include_equity_curve=True,
    include_drawdown=True,
    include_monthly_returns=True,
    include_risk_return=True,
    
    # Benchmark comparison
    benchmark_return=0.08,  # 8% annual return
    benchmark_volatility=0.15,  # 15% volatility
    benchmark_name="S&P 500"
)

custom_report = PerformanceReport(
    result,
    config=custom_config,
    initial_capital=backtest_params['initial_capital']
)

# Print verbose report
custom_report.print_report()

In [ ]:
# Show charts with benchmark
custom_report.show_charts()

## 4. Minimal Report

Create a minimal report showing only key metrics.

In [ ]:
# Minimal configuration
minimal_config = ReportConfig(
    include_returns=True,
    include_risk=True,
    include_trading=False,
    include_portfolio=False,
    include_assessment=False,
    verbose_metrics=False,
    
    # Only equity and drawdown charts
    include_equity_curve=True,
    include_drawdown=True,
    include_monthly_returns=False,
    include_risk_return=False
)

minimal_report = PerformanceReport(result, config=minimal_config)
minimal_report.print_report()

In [ ]:
# Show minimal charts
minimal_report.show_charts()

## 5. Export Metrics as Dictionary

In [ ]:
# Get metrics as dictionary
metrics_dict = report.get_metrics_dict(include_calculated=True)

# Display some key metrics
print("Key Metrics:")
print(f"Total Return: {metrics_dict['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics_dict['sharpe_ratio']:.3f}")
print(f"Max Drawdown: {metrics_dict['max_drawdown']:.2%}")
print(f"Final Value: ${metrics_dict['final_value']:,.0f}")
print(f"Profit/Loss: ${metrics_dict['profit_loss']:,.0f}")

## 6. Export Metrics as DataFrame

In [ ]:
# Get metrics as DataFrame (transposed)
metrics_df = report.get_metrics_dataframe(transpose=True, include_calculated=True)
metrics_df

In [ ]:
# Get as regular DataFrame (columns)
metrics_df_cols = report.get_metrics_dataframe(transpose=False)
metrics_df_cols

## 7. Generate Individual Charts

You can also generate and customize individual charts.

In [ ]:
# Get the figure object for further customization
fig = report.generate_charts(return_fig=True, show_plots=False)

# Customize if needed
fig.suptitle('My Custom Performance Dashboard', fontsize=18)

# Save to file
# fig.savefig('performance_report.png', dpi=300, bbox_inches='tight')

plt.show()

## 8. No Emojis Mode

For clean text output without emojis.

In [ ]:
# Print report without emojis
report.print_report(use_emojis=False)

## 9. Using in Streamlit

Here's how you would use this in a Streamlit app:

```python
import streamlit as st
from backt.reporting import PerformanceReport, ReportConfig

# After running backtest and getting result...

# Create report
report = PerformanceReport(result, initial_capital=initial_capital)

# Display text report
st.text(report.format_text_report())

# Display charts
fig = report.generate_charts(return_fig=True, show_plots=False)
st.pyplot(fig)

# Display metrics as DataFrame
st.subheader("Detailed Metrics")
st.dataframe(report.get_metrics_dataframe())
```

## Summary

The `PerformanceReport` module provides:

1. **Standardized text reports** with customizable sections
2. **Comprehensive visualizations** with flexible chart options
3. **Multiple output formats**: text, dict, DataFrame
4. **Configurable verbosity** for detailed or minimal reports
5. **Benchmark comparison** support
6. **Reusable across platforms** (Jupyter, Streamlit, CLI)

No more copy-pasting reporting code!